In [3]:
import pandas as pd

data = pd.read_csv('Zipcode.csv')
data.__len__

<bound method DataFrame.__len__ of         Zipcode   District         StateName
0        175106      MANDI  HIMACHAL PRADESH
1        175106      MANDI  HIMACHAL PRADESH
2        175123      MANDI  HIMACHAL PRADESH
3        175101      KULLU  HIMACHAL PRADESH
4        175103      KULLU  HIMACHAL PRADESH
...         ...        ...               ...
157121   250002     MEERUT     UTTAR PRADESH
157122   250002     MEERUT     UTTAR PRADESH
157123   250001     MEERUT     UTTAR PRADESH
157124   673611  KOZHIKODE            KERALA
157125   673572  KOZHIKODE            KERALA

[157126 rows x 3 columns]>

In [5]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode
import time
import numpy as np # Used for handling missing values

# --- 1. SETUP ---
# IMPORTANT: Replace with your actual API key
API_KEY = '1dc7172ab25f4f5d8bf5685c8b45e138'
geocoder = OpenCageGeocode(API_KEY)

# IMPORTANT: Adjust this if your pincode column has a different name
PINCODE_COLUMN = 'Zipcode'

# --- 2. LOAD DATASET ---
try:
    # Read the CSV file into a pandas DataFrame
    data = pd.read_csv('Zipcode.csv')
    print("✅ Successfully loaded 'Zipcode.csv'")
except FileNotFoundError:
    print("❌ Error: 'Zipcode.csv' not found. Make sure it's in the same directory as the script.")
    exit()

# --- 3. PREPARE DATAFRAME ---
# Create 'latitude' and 'longitude' columns if they don't exist
# and fill them with a null value (np.nan)
if 'latitude' not in data.columns:
    data['latitude'] = np.nan
if 'longitude' not in data.columns:
    data['longitude'] = np.nan

# --- 4. GEOCODE MISSING PINCODES ---
# Iterate only over the rows where 'latitude' is null
for index, row in data[data['latitude'].isnull()].iterrows():
    pincode = row[PINCODE_COLUMN]
    if index == 2500:
        break
    # Skip if pincode itself is invalid
    if not pincode or pd.isnull(pincode):
        print(f"⚠️ Skipping row {index} due to empty pincode.")
        continue

    try:
        # Create the search query
        query = f"{int(pincode)}, India"
        result = geocoder.geocode(query)

        if result and len(result):
            # Extract latitude and longitude from the first result
            lat = result[0]['geometry']['lat']
            lng = result[0]['geometry']['lng']

            # Update the DataFrame at the specific row and columns
            data.loc[index, 'latitude'] = lat
            data.loc[index, 'longitude'] = lng
            print(f"Updated: {pincode} → ({lat}, {lng})")
        else:
            print(f"⚠️ No result for {pincode}")

        # Respect the API's rate limit (1 request per second)
        time.sleep(1.2)

    except Exception as e:
        print(f"❌ Error with pincode {pincode}: {e}")


# --- 5. SAVE UPDATED DATASET ---
output_filename = 'Zipcode_updated.csv'
data.to_csv(output_filename, index=False)

print(f"\n🎉 Geocoding complete. The updated data has been saved to '{output_filename}'")

✅ Successfully loaded 'Zipcode.csv'
Updated: 175106 → (31.6978772, 77.1852774)
Updated: 175106 → (31.6978772, 77.1852774)
Updated: 175123 → (31.6323731, 77.3641815)
Updated: 175101 → (31.9325066, 77.1411785)
Updated: 175103 → (32.2894718, 77.1717337)
Updated: 175123 → (31.6323731, 77.3641815)
Updated: 175123 → (31.6323731, 77.3641815)
Updated: 175123 → (31.6323731, 77.3641815)
Updated: 175123 → (31.6323731, 77.3641815)
Updated: 175106 → (31.6978772, 77.1852774)
Updated: 175106 → (31.6978772, 77.1852774)
Updated: 175106 → (31.6978772, 77.1852774)
Updated: 175123 → (31.6323731, 77.3641815)
Updated: 175123 → (31.6323731, 77.3641815)
Updated: 175123 → (31.6323731, 77.3641815)
Updated: 175123 → (31.6323731, 77.3641815)
Updated: 175101 → (31.9325066, 77.1411785)
Updated: 175123 → (31.6323731, 77.3641815)
Updated: 175101 → (31.9325066, 77.1411785)
Updated: 175101 → (31.9325066, 77.1411785)
Updated: 175103 → (32.2894718, 77.1717337)
Updated: 175123 → (31.6323731, 77.3641815)
Updated: 175123 → 

In [ ]:
import pandas as pd

data = pd.read_csv('Zipcode_updated.csv')


<bound method DataFrame.__len__ of         Zipcode   District         StateName   latitude  longitude
0        175106      MANDI  HIMACHAL PRADESH  31.697877  77.185277
1        175106      MANDI  HIMACHAL PRADESH  31.697877  77.185277
2        175123      MANDI  HIMACHAL PRADESH  31.632373  77.364182
3        175101      KULLU  HIMACHAL PRADESH  31.932507  77.141178
4        175103      KULLU  HIMACHAL PRADESH  32.289472  77.171734
...         ...        ...               ...        ...        ...
157121   250002     MEERUT     UTTAR PRADESH        NaN        NaN
157122   250002     MEERUT     UTTAR PRADESH        NaN        NaN
157123   250001     MEERUT     UTTAR PRADESH        NaN        NaN
157124   673611  KOZHIKODE            KERALA        NaN        NaN
157125   673572  KOZHIKODE            KERALA        NaN        NaN

[157126 rows x 5 columns]>

In [8]:
import pandas as pd

data = pd.read_csv('Zipcode_updated.csv')
print("Initial number of entries:", len(data))
# Remove duplicate Zipcode entries
data = data.drop_duplicates(subset='Zipcode', keep='first')

# (Optional) Save cleaned data
data.to_csv('Zipcode_deduplicated.csv', index=False)

print("✅ Duplicates removed. Unique pincodes:", len(data))

Initial number of entries: 157126
✅ Duplicates removed. Unique pincodes: 19300
